# Vorlesung Syntaktische & Semantische Interoperabilität

In [ ]:
# Installation (ggf. nach ersten Lauf auskommentieren)
#!pip install folium
#!pip install altair

# Libraries
import numpy
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import altair as alt

# Konfiguration
base_url = "https://raw.githubusercontent.com/ganslats/interoperability-excercise/main/data"

## Patientenstammdaten

In [ ]:
# Patientenstammsätze laden
patients_mannheim = pd.read_csv(base_url + "/Mannheim/patients.csv", index_col=0)
patients_leipzig  = pd.read_csv(base_url + "/Leipzig/patients.csv",  index_col=0)

# Patientenstammsätze zusammenführen
patients = pd.concat([patients_mannheim, patients_leipzig])

# Karte mit Fokus Bundesgebiet erzeugen
map = folium.Map(location=[51.151509, 10.217850], zoom_start=6)
marker_cluster = MarkerCluster().add_to(map)

# Patientenlokalisationen hinzufügen
for row in patients.itertuples():
    folium.Marker(location=[row.LAT,row.LON]).add_to(marker_cluster)

# Karte anzeigen
map

## Befunddaten

In [ ]:
# Befunddaten laden
observations_mannheim = pd.read_csv(base_url + "/Mannheim/observations-glucose.csv", index_col = False)
observations_leipzig  = pd.read_csv(base_url + "/Leipzig/observations-glucose.csv",  index_col = False)

# Ortsnamen hinzufügen
observations_mannheim["ORT"] = "Mannheim"
observations_leipzig["ORT"]  = "Leipzig"

### Befunddaten Mannheim

In [ ]:
observations_mannheim

### Befunddaten Leipzig

In [ ]:
observations_leipzig

## Syntaktisches Mapping, Zusammenführung & Exploration der Daten

In [ ]:
# Spaltennamen Mannheim für Leipzig übernehmen
observations_leipzig.rename({'PATNR':       'PATIENT_ID',
                             'FALNR':       'FALL_ID', 
                             'DATUM':       'DATUM_ZEIT', 
                             'ANALYT_CODE': 'METHODE_ID', 
                             'ANALYT_NAME': 'METHODE_LANGNAME'}, axis = 1, inplace = True)

# Befunddaten von Mannheim & Leipzig zusammenführen
observations = pd.concat([observations_mannheim, observations_leipzig])

### Zusammengeführter Befunddatensatz

In [ ]:
# Zusammengeführte Befunddaten ausgeben
observations

### Boxplot der Befunddaten nach Standort

In [ ]:
# Boxplot der zusammengeführten Daten ausgeben
alt.data_transformers.disable_max_rows()
alt.Chart(observations).mark_boxplot().encode(
    y     = alt.Y('WERT', axis=alt.Axis(title='Wert')),
    facet = alt.Facet('ORT', title=None)
).properties(width=100)

## Semantisches Mapping, Harmonisierung & erneute Exploration der Daten

### Distinkte Ausprägungen der Laboranalyte

In [ ]:
# Zusammenfassung der Analyte & Einheiten nach Standort
observations.groupby(['ORT', 'METHODE_ID', 'METHODE_LANGNAME', 'EINHEIT'], as_index=False).agg({'WERT': 'count'})

### Mappingtabelle 

In [ ]:
# Mappingtabelle erzeugen
mapping = pd.DataFrame({"METHODE_ID":         ["BLZ-STAT", "BLZ-ZLAB", "GLUPO",   "GLUSM"],
                        "HARM_LOINC_CODE":    ["14743-9",  "14749-6",  "41653-7", "2345-7"],
                        "HARM_LOINC_NAME":    ["Glucose [Moles/volume] in Capillary blood by Glucometer",
                                               "Glucose [Moles/volume] in Serum or Plasma",
                                               "Glucose [Mass/volume] in Capillary blood by Glucometer",
                                               "Glucose [Mass/volume] in Serum or Plasma"],
                        "HARM_KURZNAME":      ["Glucose POCT", "Glucose Zentrallabor", "Glucose POCT", "Glucose Zentrallabor"],
                        "HARM_EINHEIT":       ["mg/dl", "mg/dl", "mg/dl", "mg/dl"],
                        "HARM_MULTIPLIKATOR": [18.02, 18.02, 1, 1]})

mapping

### Mapping anwenden und Maßeinheiten harmonisieren

In [ ]:
# Befunddaten & Mapping verknüpfen
harm_observations = pd.merge(observations, mapping, on="METHODE_ID")

# Befundwerte harmonisieren
harm_observations["HARM_WERT"] = harm_observations["WERT"] * harm_observations["HARM_MULTIPLIKATOR"]

# Boxplot der harmonisierten Befundwerte ausgeben
alt.data_transformers.disable_max_rows()
alt.Chart(harm_observations).mark_boxplot().encode(
    x     = alt.X('HARM_KURZNAME', axis=alt.Axis(title='')),
    y     = alt.Y('HARM_WERT', axis=alt.Axis(title='Blutglucose [mg/dL]')),
    color = alt.Color('HARM_KURZNAME', legend=alt.Legend(title="Untersuchung")),
    facet = alt.Facet('ORT', title=None)
).properties(width=100)